In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import cross_val_score, RandomizedSearchCV
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")

print(tf.test.is_gpu_available())

train_df = pd.read_csv('task3_2021.training.csv')
test_df = pd.read_csv('task3_2021.test.csv')
n_pre_train_y = train_df['target']
n_pre_train_x = train_df.drop(['target'], axis=1)
n_pre_test_x = test_df
# display(n_pre_train_x.head(), n_pre_train_y.head(), n_pre_train_y.nunique(), n_pre_test_x.head())

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
True
time: 1.9 s (started: 2021-12-21 15:00:41 +00:00)


2021-12-21 15:00:42.732348: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-21 15:00:42.903653: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-21 15:00:42.904107: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-21 15:00:42.907241: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [2]:
def oh_pre(x):
    for k in ['f15', 'f16', 'f17']:
        tmp_one_hot_k =  pd.get_dummies(x[k], prefix=f"{k}")
        x = pd.concat([x, tmp_one_hot_k],axis=1)
        x = x.drop([k], axis=1)
    return x

oh_pre_train_x = oh_pre(n_pre_train_x)
oh_pre_test_x = oh_pre(n_pre_test_x)
scaler = StandardScaler()
train_x = scaler.fit_transform(oh_pre_train_x)
test_x = scaler.transform(oh_pre_test_x)

ohenc = OneHotEncoder(handle_unknown="ignore", sparse=False)
train_y = ohenc.fit_transform(n_pre_train_y.values.reshape(-1,1))
# mse = make_scorer(mean_squared_error, squared=False)nunique

time: 17.1 ms (started: 2021-12-21 15:00:43 +00:00)


In [7]:
from tensorflow.keras import regularizers

class DNN(tf.keras.Model):
  def __init__(self, regular_coef):
    super(DNN,self).__init__()
    self.dense1 = tf.keras.layers.Dense(8, activation=tf.nn.relu,
                                        kernel_regularizer=regularizers.l1_l2(l1=regular_coef, l2=regular_coef),
                                        bias_regularizer=regularizers.l2(regular_coef),
                                        activity_regularizer=regularizers.l2(regular_coef))
    self.dense2 = tf.keras.layers.Dense(2, activation=tf.nn.relu,
                                        kernel_regularizer=regularizers.l1_l2(l1=regular_coef, l2=regular_coef),
                                        bias_regularizer=regularizers.l2(regular_coef),
                                        activity_regularizer=regularizers.l2(regular_coef))
    self.dense3 = tf.keras.layers.Dense(48, activation=tf.nn.relu)
    self.output_layer = tf.keras.layers.Dense(10, activation=tf.nn.softmax)

  def call(self, inputs):
    x = self.dense1(inputs)
    x = self.dense2(x)
    # x = self.dense3(x)
    return self.output_layer(x)

def scheduler(epoch, lr):
    if epoch < 100:
        return lr
    else:
        return lr*tf.math.exp(-1e-2)


callback_early = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=100,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=True,
)

callback_lr = tf.keras.callbacks.LearningRateScheduler(scheduler)

batch_size = 50
epochs = 5000
opt = tf.keras.optimizers.Adam(learning_rate=1e-3)
model = DNN(1e-9)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
model.fit(train_x, train_y, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=[callback_lr,callback_early], verbose=1)
model.summary()

Epoch 1/5000
180/180 [==============================] - 1s 2ms/step - loss: 2.2779 - accuracy: 0.1284 - val_loss: 2.2195 - val_accuracy: 0.1620 - lr: 0.0010
Epoch 2/5000
180/180 [==============================] - 0s 1ms/step - loss: 2.1224 - accuracy: 0.1952 - val_loss: 2.0128 - val_accuracy: 0.2150 - lr: 0.0010
Epoch 3/5000
180/180 [==============================] - 0s 1ms/step - loss: 1.8954 - accuracy: 0.2322 - val_loss: 1.7642 - val_accuracy: 0.2330 - lr: 0.0010
Epoch 4/5000
180/180 [==============================] - 0s 1ms/step - loss: 1.6609 - accuracy: 0.2310 - val_loss: 1.5718 - val_accuracy: 0.2590 - lr: 0.0010
Epoch 5/5000
180/180 [==============================] - 0s 1ms/step - loss: 1.5121 - accuracy: 0.2579 - val_loss: 1.4571 - val_accuracy: 0.2950 - lr: 0.0010
Epoch 6/5000
180/180 [==============================] - 0s 1ms/step - loss: 1.4182 - accuracy: 0.3038 - val_loss: 1.3783 - val_accuracy: 0.3460 - lr: 0.0010
Epoch 7/5000
180/180 [==============================] - 0s

In [ ]:
from pandas import RangeIndex

nn_pred = model.predict(test_x)
for i in range(9,0,-1):
    thres = i/10
    pred = np.where(nn_pred<thres, 0, 1)
    output = ohenc.inverse_transform(pred)
    check = (output==None).sum()
    print(f"threshold={thres} => Number of None is {check}")
    if check==0:break

if check==0:
    pd.Series(output.reshape(-1), index=RangeIndex(1, 2001), name='Predicted').to_csv('ans_nn_6.csv',index_label='id')